In [3]:
#Check version
import bokeh
print(bokeh.__version__)

3.7.0


In [2]:
##Working version with label names (another version below):
# === FINAL DASHBOARD: Dropdown + Selection + Axis Labels ===
import panel as pn
import pandas as pd
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure
pn.extension('bokeh')

# --- Load & clean ---
df = pd.read_excel("../data/DATA_UFM_combined.xlsx")
plot_cols = ['tidsforbrug_p50', 'ensom_likert', 'arbejdstid_timer', 'maanedloen_10aar', 'titel']

df['hovedinsttx'] = df['hovedinsttx'].astype(str).str.strip()
df = df[df['hovedinsttx'].str.lower() != 'nan']
df_clean = df.dropna(subset=plot_cols + ['hovedinsttx'])

categories = sorted(df_clean['hovedinsttx'].unique())

# --- Dropdown ---
dropdown = pn.widgets.Select(name="Hovedinstitution", options=categories, value=categories[0])

# --- Sources ---
source_main = ColumnDataSource(df_clean[df_clean['hovedinsttx'] == categories[0]][plot_cols])
source_selected = ColumnDataSource(data={c: [] for c in plot_cols})

# --- Plot 1: Tidsforbrug vs Ensomhed ---
p1 = figure(
    title="Tidsforbrug vs Ensomhed",
    width=500, height=400,
    tools="pan,wheel_zoom,box_select,lasso_select,reset",
    tooltips=[
        ("Tidsforbrug", "@tidsforbrug_p50"),
        ("Ensomhed", "@ensom_likert"),
        ("Titel", "@titel")
    ]
)
p1.circle('tidsforbrug_p50', 'ensom_likert', size=8, source=source_main,
          selection_color="orange", nonselection_alpha=0.3)

# AXIS LABELS
p1.xaxis.axis_label = "Tidsforbrug (p50)"
p1.yaxis.axis_label = "Ensomhed (Likert)"

# --- Plot 2: Arbejdstid vs Månedsløn (valgte) ---
p2 = figure(
    title="Valgte: Arbejdstid vs Månedsløn",
    width=500, height=400,
    tooltips=[
        ("Timer", "@arbejdstid_timer"),
        ("Løn", "@maanedloen_10aar"),
        ("Titel", "@titel")
    ]
)
p2.square('arbejdstid_timer', 'maanedloen_10aar', size=8, color="red", source=source_selected)

# AXIS LABELS
p2.xaxis.axis_label = "Arbejdstid (timer/uge)"
p2.yaxis.axis_label = "Månedsløn (10-års niveau)"

# --- Dropdown update ---
@pn.depends(dropdown.param.value, watch=True)
def update_main_plot(cat):
    filtered = df_clean[df_clean['hovedinsttx'] == cat][plot_cols]
    source_main.data = filtered.to_dict('list')
    source_selected.data = {c: [] for c in plot_cols}
    p1.title.text = f"{10}"

# --- Selection update ---
def update_selected(event=None):
    indices = source_main.selected.indices
    if indices:
        selected_df = pd.DataFrame(source_main.data).iloc[indices]
        source_selected.data = selected_df[plot_cols].to_dict('list')
    else:
        source_selected.data = {c: [] for c in plot_cols}

source_main.selected.on_change('indices', lambda attr, old, new: update_selected())

# --- Layout ---
dashboard = pn.Column(
    dropdown,
    pn.Row(p1, p2)
)

dashboard.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'c0cdf965-f180-4e71-a232-5c96364473f9': {'version…

In [ ]:
#Bidirectional highlighting
# === FINAL: BI-DIRECTIONAL + TOOLBARS VISIBLE + NO ERRORS ===
import panel as pn
import pandas as pd
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure

pn.extension('bokeh')

# --- Load & clean ---
df = pd.read_excel("../data/DATA_UFM_combined.xlsx")
plot_cols = ['tidsforbrug_p50', 'ensom_likert', 'arbejdstid_timer', 'maanedloen_10aar', 'titel']

df['hovedinsttx'] = df['hovedinsttx'].astype(str).str.strip()
df = df[df['hovedinsttx'].str.lower() != 'nan']
df_clean = df.dropna(subset=plot_cols + ['hovedinsttx'])

categories = sorted(df_clean['hovedinsttx'].unique())

# --- Dropdown ---
dropdown = pn.widgets.Select(name="Hovedinstitution", options=categories, value=categories[0])

# --- SHARED SOURCE ---
source = ColumnDataSource(df_clean[df_clean['hovedinsttx'] == categories[0]][plot_cols])

# --- Plot 1 ---
p1 = figure(
    title="Tidsforbrug vs Ensomhed",
    width=500, height=400,
    tools="pan,wheel_zoom,box_select,lasso_select,reset",  # ← FIXED
    toolbar_location="above",
    tooltips=[
        ("Tidsforbrug", "@tidsforbrug_p50"),
        ("Ensomhed", "@ensom_likert"),
        ("Titel", "@titel")
    ]
)
p1.circle('tidsforbrug_p50', 'ensom_likert', size=8, source=source,
          selection_color="orange", nonselection_alpha=0.3)

p1.xaxis.axis_label = "Tidsforbrug (p50)"
p1.yaxis.axis_label = "Ensomhed (Likert)"

# --- Plot 2 ---
p2 = figure(
    title="Arbejdstid vs Månedsløn",
    width=500, height=400,
    tools="pan,wheel_zoom,box_select,lasso_select,reset",  # ← FIXED
    toolbar_location="above",
    tooltips=[
        ("Timer", "@arbejdstid_timer"),
        ("Løn", "@maanedloen_10aar"),
        ("Titel", "@titel")
    ]
)
p2.circle('arbejdstid_timer', 'maanedloen_10aar', size=8, source=source,
          selection_color="orange", nonselection_alpha=0.3)

p2.xaxis.axis_label = "Arbejdstid (timer/uge)"
p2.yaxis.axis_label = "Månedsløn (10-års niveau)"

# --- Dropdown update ---
@pn.depends(dropdown.param.value, watch=True)
def update_plots(cat):
    filtered = df_clean[df_clean['hovedinsttx'] == cat][plot_cols]
    source.data = filtered.to_dict('list')
    p1.title.text = f"{cat} ({len(filtered)} points)"
    p2.title.text = f"{cat} ({len(filtered)} points)"

# --- BI-DIRECTIONAL SELECTION (auto via shared source) ---
source.selected.on_change('indices', lambda attr, old, new: None)

# --- Layout ---
dashboard = pn.Column(
    dropdown,
    pn.Row(p1, p2, sizing_mode="stretch_both")
)

dashboard.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'0c8c9e22-9c58-4cc4-a2c8-432ac46f6980': {'version…

In [ ]:
#Same as above, but html version so it's fully visible. Prints to dashboard.html:
# === SAVE AS HTML: Full Interactive Dashboard in Browser ===
import panel as pn
import pandas as pd
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure

pn.extension('bokeh')

# --- Load & clean ---
df = pd.read_excel("../data/DATA_UFM_combined.xlsx")
plot_cols = ['tidsforbrug_p50', 'ensom_likert', 'arbejdstid_timer', 'maanedloen_10aar', 'titel']

df['hovedinsttx'] = df['hovedinsttx'].astype(str).str.strip()
df = df[df['hovedinsttx'].str.lower() != 'nan']
df_clean = df.dropna(subset=plot_cols + ['hovedinsttx'])

categories = sorted(df_clean['hovedinsttx'].unique())

# --- Dropdown ---
dropdown = pn.widgets.Select(name="Hovedinstitution", options=categories, value=categories[0])

# --- SHARED SOURCE ---
source = ColumnDataSource(df_clean[df_clean['hovedinsttx'] == categories[0]][plot_cols])

# --- Plot 1 ---
p1 = figure(
    title="Tidsforbrug vs Ensomhed",
    width=600, height=500,  # ← Bigger for browser
    tools="pan,wheel_zoom,box_select,lasso_select,reset",
    toolbar_location="above",
    tooltips=[
        ("Tidsforbrug", "@tidsforbrug_p50"),
        ("Ensomhed", "@ensom_likert"),
        ("Titel", "@titel")
    ]
)
p1.circle('tidsforbrug_p50', 'ensom_likert', size=8, source=source,
          selection_color="orange", nonselection_alpha=0.3)

p1.xaxis.axis_label = "Tidsforbrug (p50)"
p1.yaxis.axis_label = "Ensomhed (Likert)"

# --- Plot 2 ---
p2 = figure(
    title="Arbejdstid vs Månedsløn",
    width=600, height=500,  # ← Bigger
    tools="pan,wheel_zoom,box_select,lasso_select,reset",
    toolbar_location="above",
    tooltips=[
        ("Timer", "@arbejdstid_timer"),
        ("Løn", "@maanedloen_10aar"),
        ("Titel", "@titel")
    ]
)
p2.circle('arbejdstid_timer', 'maanedloen_10aar', size=8, source=source,
          selection_color="orange", nonselection_alpha=0.3)

p2.xaxis.axis_label = "Arbejdstid (timer/uge)"
p2.yaxis.axis_label = "Månedsløn (10-års niveau)"

# --- Dropdown update ---
@pn.depends(dropdown.param.value, watch=True)
def update_plots(cat):
    filtered = df_clean[df_clean['hovedinsttx'] == cat][plot_cols]
    source.data = filtered.to_dict('list')
    p1.title.text = f"{cat} ({len(filtered)} points)"
    p2.title.text = f"{cat} ({len(filtered)} points)"

# --- BI-DIRECTIONAL SELECTION ---
source.selected.on_change('indices', lambda attr, old, new: None)

# --- Layout ---
dashboard = pn.Column(
    dropdown,
    pn.Row(p1, p2, sizing_mode="stretch_both"),
    sizing_mode="stretch_width"
)

# --- SAVE TO HTML (FULL BROWSER EXPERIENCE) ---
dashboard.save("dashboard.html", embed=True)

print("Dashboard saved to 'dashboard.html'")
print("Open it in your browser for full interactive view!")

Dashboard saved to 'dashboard.html'             
Open it in your browser for full interactive view!


In [3]:
#This version apps a clickable map, that toggles which institutions you can see
#It also matches that logic to the plots - i.e DMJX has programs in both Aarhus and KBH
#But it only shows the ones relevant for what you've clicked on
#It also adds tap to highlight, and holding shift allows you to highlight more points

import pandas as pd
import requests
import json
from bokeh.models import (
    ColumnDataSource, GeoJSONDataSource, HoverTool, CustomJS, Button, Select, TapTool
)
from bokeh.plotting import figure, output_file, save
from bokeh.layouts import column, row

# --- Load data ---
df = pd.read_excel("../data/DATA_UFM_combined.xlsx")
plot_cols = [
    'tidsforbrug_p50', 'ensom_likert', 'arbejdstid_timer',
    'maanedloen_10aar', 'hovedinsttx', 'instkommunetx', 'titel'
]
df_clean = df.dropna(subset=plot_cols)

# Normalize strings
df_clean['instkommunetx'] = df_clean['instkommunetx'].astype(str).str.strip()
df_clean['hovedinsttx'] = df_clean['hovedinsttx'].astype(str).str.strip()

# All institutions
all_institutions = sorted(df_clean['hovedinsttx'].unique())

# Kommuner present in dataset (lowercase)
kommuner_in_data = set(df_clean['instkommunetx'].str.lower())

# --- Fetch GeoJSON (all kommuner) ---
response = requests.get("https://api.dataforsyningen.dk/kommuner?format=geojson")
geojson_data = response.json()

# Annotate features
for feat in geojson_data['features']:
    navn = feat['properties'].get('navn', '').strip().lower()
    clickable = navn in kommuner_in_data
    feat['properties']['clickable'] = bool(clickable)
    feat['properties']['selected'] = False
    feat['properties']['fill_color'] = 'lightblue' if clickable else 'lightgrey'

geojson_str = json.dumps(geojson_data)

# --- Data sources ---
full_data = ColumnDataSource(df_clean)        # master copy for filtering
source_scatter = ColumnDataSource(df_clean)   # scatter plot data
source_municipalities = GeoJSONDataSource(geojson=geojson_str)

# --- Widgets ---
dropdown = Select(title="Hovedinstitution", value=all_institutions[0], options=all_institutions)
reset_button = Button(label="Reset All", button_type="success", width=100)

# --- Map plot ---
map_plot = figure(
    title="Click a Kommune to Filter Institutions (toggle multi-select)",
    width=600, height=600,
    tools="tap,reset",
    toolbar_location="above"
)

patches = map_plot.patches(
    'xs', 'ys', source=source_municipalities,
    fill_color='fill_color',
    line_color="blue", line_width=1.0,
    fill_alpha=0.85,
    nonselection_alpha=0.4
)
hover_map = HoverTool(renderers=[patches], tooltips=[("Kommune", "@navn")])
map_plot.add_tools(hover_map)

# --- Scatter plots with hover tooltips + tap multi-select ---
hover_tool_p1 = HoverTool(tooltips=[
    ("Tidsforbrug", "@tidsforbrug_p50"),
    ("Ensomhed", "@ensom_likert"),
    ("Titel", "@titel"),
    ("Kommune", "@instkommunetx")
])
hover_tool_p2 = HoverTool(tooltips=[
    ("Timer", "@arbejdstid_timer"),
    ("Løn", "@maanedloen_10aar"),
    ("Titel", "@titel"),
    ("Kommune", "@instkommunetx")
])

p1 = figure(
    title="Tidsforbrug vs Ensomhed",
    width=400, height=400,
    tools="pan,wheel_zoom,box_select,lasso_select,reset",
    toolbar_location="above"
)
p1.add_tools(hover_tool_p1)
tap1 = TapTool()  # default: Shift+click allows multi-selection
p1.add_tools(tap1)
p1.circle('tidsforbrug_p50', 'ensom_likert', size=8, source=source_scatter,
          selection_color="orange", nonselection_alpha=0.3)

p2 = figure(
    title="Arbejdstid vs Månedsløn",
    width=400, height=400,
    tools="pan,wheel_zoom,box_select,lasso_select,reset",
    toolbar_location="above"
)
p2.add_tools(hover_tool_p2)
tap2 = TapTool()
p2.add_tools(tap2)
p2.circle('arbejdstid_timer', 'maanedloen_10aar', size=8, source=source_scatter,
          selection_color="orange", nonselection_alpha=0.3)

# --- JS: map selection callback (cross-filter + highlight) ---
map_callback = CustomJS(args=dict(
    source_scatter=source_scatter,
    source_municipalities=source_municipalities,
    dropdown=dropdown,
    full_data=full_data
), code="""
    const geojson = JSON.parse(source_municipalities.geojson);
    const inds = cb_obj.indices;

    // Toggle selected kommuner (Shift behavior automatic handled in Bokeh)
    inds.forEach(i => {
        const feat = geojson.features[i];
        if (feat.properties.clickable) {
            feat.properties.selected = !feat.properties.selected;
        }
    });

    // Update fill colors
    geojson.features.forEach(f => {
        if (!f.properties.clickable) {
            f.properties.fill_color = 'lightgrey';
        } else if (f.properties.selected) {
            f.properties.fill_color = 'orange';
        } else {
            f.properties.fill_color = 'lightblue';
        }
    });
    source_municipalities.geojson = JSON.stringify(geojson);

    // Get selected kommuner
    const selected_kommuner = geojson.features
        .filter(f => f.properties.selected)
        .map(f => f.properties.navn.toLowerCase());

    const data_all = full_data.data;
    const keys = Object.keys(data_all);

    // Cross-filter by both dropdown and selected kommuner
    const selected_inst = dropdown.value;
    const keep = [];
    for (let i=0; i<data_all['instkommunetx'].length; i++) {
        const row_komm = (data_all['instkommunetx'][i] || '').toLowerCase();
        const row_inst = data_all['hovedinsttx'][i];
        if ((selected_kommuner.length === 0 || selected_kommuner.includes(row_komm)) &&
            row_inst === selected_inst) keep.push(i);
    }

    // Rebuild filtered data
    const new_data = {};
    for (let k=0; k<keys.length; k++) {
        const key = keys[k];
        new_data[key] = keep.map(i => data_all[key][i]);
    }
    source_scatter.data = new_data;

    // Update dropdown to only institutions in selected kommuner
    const inst_set = Array.from(new Set(
        data_all['hovedinsttx'].filter((val,i) => 
            selected_kommuner.length===0 || selected_kommuner.includes((data_all['instkommunetx'][i]||'').toLowerCase()))
    )).sort();
    dropdown.options = inst_set;
    if (inst_set.length>0 && !inst_set.includes(selected_inst)) dropdown.value = inst_set[0];

    source_scatter.change.emit();
""")
patches.data_source.selected.js_on_change('indices', map_callback)

# --- JS: dropdown callback (cross-filter by selected kommuner) ---
dropdown_callback = CustomJS(args=dict(
    source_scatter=source_scatter,
    full_data=full_data,
    source_municipalities=source_municipalities
), code="""
    const val = cb_obj.value;
    const data_all = full_data.data;
    const keys = Object.keys(data_all);

    // Selected kommuner from map
    const geojson = JSON.parse(source_municipalities.geojson);
    const selected_kommuner = geojson.features
        .filter(f => f.properties.selected)
        .map(f => f.properties.navn.toLowerCase());

    const keep = [];
    for (let i=0; i<data_all['hovedinsttx'].length; i++) {
        const row_inst = data_all['hovedinsttx'][i];
        const row_komm = (data_all['instkommunetx'][i] || '').toLowerCase();
        if (row_inst === val && (selected_kommuner.length===0 || selected_kommuner.includes(row_komm))) keep.push(i);
    }

    const new_data = {};
    for (let k=0; k<keys.length; k++) {
        const key = keys[k];
        new_data[key] = keep.map(i => data_all[key][i]);
    }
    source_scatter.data = new_data;
    source_scatter.change.emit();
""")
dropdown.js_on_change('value', dropdown_callback)

# --- Reset button JS (resets selection + zoom) ---
reset_callback = CustomJS(args=dict(
    source_scatter=source_scatter,
    source_municipalities=source_municipalities,
    dropdown=dropdown,
    full_data=full_data,
    p1=p1, p2=p2
), code="""
    // Reset map
    const geojson = JSON.parse(source_municipalities.geojson);
    geojson.features.forEach(f => {
        f.properties.selected = false;
        f.properties.fill_color = f.properties.clickable ? 'lightblue' : 'lightgrey';
    });
    source_municipalities.geojson = JSON.stringify(geojson);

    // Reset scatter plots
    const data_all = full_data.data;
    source_scatter.data = Object.assign({}, data_all);

    // Reset dropdown
    const all_insts = Array.from(new Set(data_all['hovedinsttx'])).sort();
    dropdown.options = all_insts;
    dropdown.value = all_insts[0];

    // Reset zoom ranges
    p1.reset.emit();
    p2.reset.emit();

    source_scatter.change.emit();
""")
reset_button.js_on_click(reset_callback)

# --- Layout & save ---
layout = column(row(dropdown, reset_button), row(map_plot, column(p1, p2)))
output_file("kommune_dashboard.html")
save(layout)
print("Dashboard saved to 'dashboard_map.html' — multiselect, hover, tap multi-select, cross-filtered")


C:\Users\whart\AppData\Local\Temp\ipykernel_24088\1976824772.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['instkommunetx'] = df_clean['instkommunetx'].astype(str).str.strip()
C:\Users\whart\AppData\Local\Temp\ipykernel_24088\1976824772.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['hovedinsttx'] = df_clean['hovedinsttx'].astype(str).str.strip()


Dashboard saved to 'dashboard_map.html' — multiselect, hover, tap multi-select, cross-filtered


In [5]:
#This version adds search for programs
#Logic is build such that you can only search for programs visible in the plot
#Seaching then highlights the program, using the same logic as clicking on it
#Searching for more items, adds them to the highlighted items
#Clicking (unless you hold shift) then deselects, just like box/lasso select also does
#Hence it hopefully works as expected
import pandas as pd
import requests
import json
from bokeh.models import (
    ColumnDataSource, GeoJSONDataSource, HoverTool, CustomJS, Button, Select, TapTool,
    AutocompleteInput
)
from bokeh.plotting import figure, output_file, save
from bokeh.layouts import column, row

# --- Load data ---
df = pd.read_excel("../data/DATA_UFM_combined.xlsx")
plot_cols = [
    'tidsforbrug_p50', 'ensom_likert', 'arbejdstid_timer',
    'maanedloen_10aar', 'hovedinsttx', 'instkommunetx', 'titel'
]
df_clean = df.dropna(subset=plot_cols)

# Normalize strings
df_clean['instkommunetx'] = df_clean['instkommunetx'].astype(str).str.strip()
df_clean['hovedinsttx'] = df_clean['hovedinsttx'].astype(str).str.strip()
df_clean['titel'] = df_clean['titel'].astype(str).str.strip()

# All institutions
all_institutions = sorted(df_clean['hovedinsttx'].unique())

# Kommuner present in dataset (lowercase)
kommuner_in_data = set(df_clean['instkommunetx'].str.lower())

# --- Fetch GeoJSON (all kommuner) ---
response = requests.get("https://api.dataforsyningen.dk/kommuner?format=geojson")
geojson_data = response.json()

# Annotate features
for feat in geojson_data['features']:
    navn = feat['properties'].get('navn', '').strip().lower()
    clickable = navn in kommuner_in_data
    feat['properties']['clickable'] = bool(clickable)
    feat['properties']['selected'] = False
    feat['properties']['fill_color'] = 'lightblue' if clickable else 'lightgrey'

geojson_str = json.dumps(geojson_data)

# --- Data sources ---
full_data = ColumnDataSource(df_clean)        # master copy
source_scatter = ColumnDataSource(df_clean)   # scatter plot data
source_municipalities = GeoJSONDataSource(geojson=geojson_str)

# --- Widgets ---
dropdown = Select(title="Hovedinstitution", value=all_institutions[0], options=all_institutions)
reset_button = Button(label="Reset All", button_type="success", width=100)
search_input = AutocompleteInput(title="Search Titel:", completions=[], min_characters=0, placeholder="Search to highlight...")

# --- Map plot ---
map_plot = figure(
    title="Click a Kommune to Filter Institutions (toggle multi-select)",
    width=600, height=600,
    tools="tap,reset",
    toolbar_location="above"
)
patches = map_plot.patches(
    'xs', 'ys', source=source_municipalities,
    fill_color='fill_color',
    line_color="blue", line_width=1.0,
    fill_alpha=0.85,
    nonselection_alpha=0.4
)
hover_map = HoverTool(renderers=[patches], tooltips=[("Kommune", "@navn")])
map_plot.add_tools(hover_map)

# --- Scatter plots ---
hover_tool_p1 = HoverTool(tooltips=[("Tidsforbrug", "@tidsforbrug_p50"),
                                    ("Ensomhed", "@ensom_likert"),
                                    ("Titel", "@titel"),
                                    ("Kommune", "@instkommunetx")])
hover_tool_p2 = HoverTool(tooltips=[("Timer", "@arbejdstid_timer"),
                                    ("Løn", "@maanedloen_10aar"),
                                    ("Titel", "@titel"),
                                    ("Kommune", "@instkommunetx")])

p1 = figure(title="Tidsforbrug vs Ensomhed", width=400, height=400,
            tools="pan,wheel_zoom,box_select,lasso_select,reset",
            toolbar_location="above")
p1.add_tools(hover_tool_p1)
tap1 = TapTool()
p1.add_tools(tap1)
p1_glyph = p1.circle('tidsforbrug_p50', 'ensom_likert', size=8, source=source_scatter,
          selection_color="orange", nonselection_alpha=0.3)

p2 = figure(title="Arbejdstid vs Månedsløn", width=400, height=400,
            tools="pan,wheel_zoom,box_select,lasso_select,reset",
            toolbar_location="above")
p2.add_tools(hover_tool_p2)
tap2 = TapTool()
p2.add_tools(tap2)
p2_glyph = p2.circle('arbejdstid_timer', 'maanedloen_10aar', size=8, source=source_scatter,
          selection_color="orange", nonselection_alpha=0.3)

# --- JS: Map callback (cross-filter + highlight) ---
map_callback = CustomJS(args=dict(
    source_scatter=source_scatter,
    source_municipalities=source_municipalities,
    dropdown=dropdown,
    search_input=search_input,
    full_data=full_data
), code="""
const geojson = JSON.parse(source_municipalities.geojson);
const inds = cb_obj.indices;

// Toggle selected kommuner
inds.forEach(i => {
    const feat = geojson.features[i];
    if (feat.properties.clickable) feat.properties.selected = !feat.properties.selected;
});

// Update fill colors
geojson.features.forEach(f => {
    if (!f.properties.clickable) f.properties.fill_color = 'lightgrey';
    else if (f.properties.selected) f.properties.fill_color = 'orange';
    else f.properties.fill_color = 'lightblue';
});
source_municipalities.geojson = JSON.stringify(geojson);

// Get selected kommuner
const selected_kommuner = geojson.features
    .filter(f => f.properties.selected)
    .map(f => f.properties.navn.toLowerCase());

const data_all = full_data.data;
const keys = Object.keys(data_all);
const selected_inst = dropdown.value;

const keep = [];
for (let i=0; i<data_all['instkommunetx'].length; i++){
    const row_komm = (data_all['instkommunetx'][i] || '').toLowerCase();
    const row_inst = data_all['hovedinsttx'][i];
    if ((selected_kommuner.length===0 || selected_kommuner.includes(row_komm)) &&
        row_inst === selected_inst) keep.push(i);
}

// Rebuild filtered data
const new_data = {};
for (let k=0; k<keys.length; k++){
    const key = keys[k];
    new_data[key] = keep.map(i => data_all[key][i]);
}
source_scatter.data = new_data;

// Update dropdown options
const inst_set = Array.from(new Set(
    data_all['hovedinsttx'].filter((val,i) =>
        selected_kommuner.length===0 || selected_kommuner.includes((data_all['instkommunetx'][i]||'').toLowerCase()))
)).sort();
dropdown.options = inst_set;
if (inst_set.length>0 && !inst_set.includes(selected_inst)) dropdown.value = inst_set[0];

// Update search completions to match current plot
const titles = Array.from(new Set(new_data['titel'] || [])).sort();
search_input.completions = titles;

source_scatter.change.emit();
""")
patches.data_source.selected.js_on_change('indices', map_callback)

# --- JS: Dropdown callback (cross-filter by map selection) ---
dropdown_callback = CustomJS(args=dict(
    source_scatter=source_scatter,
    full_data=full_data,
    source_municipalities=source_municipalities,
    search_input=search_input
), code="""
const val = cb_obj.value;
const data_all = full_data.data;
const keys = Object.keys(data_all);

// Map selected kommuner
const geojson = JSON.parse(source_municipalities.geojson);
const selected_kommuner = geojson.features
    .filter(f => f.properties.selected)
    .map(f => f.properties.navn.toLowerCase());

const keep = [];
for (let i=0; i<data_all['hovedinsttx'].length; i++){
    const row_inst = data_all['hovedinsttx'][i];
    const row_komm = (data_all['instkommunetx'][i]||'').toLowerCase();
    if (row_inst === val && (selected_kommuner.length===0 || selected_kommuner.includes(row_komm))) keep.push(i);
}

// Rebuild filtered data
const new_data = {};
for (let k=0; k<keys.length; k++){
    const key = keys[k];
    new_data[key] = keep.map(i => data_all[key][i]);
}
source_scatter.data = new_data;

// Update search completions to match current plot
const titles = Array.from(new Set(new_data['titel'] || [])).sort();
search_input.completions = titles;

source_scatter.change.emit();
""")
dropdown.js_on_change('value', dropdown_callback)

# --- JS: Search selects items (same as box/lasso/tap) ---
search_callback = CustomJS(args=dict(source_scatter=source_scatter, search_input=search_input), code="""
const sel_title = search_input.value.trim();
if (!sel_title) return;

const data = source_scatter.data;
const indices = [];
for (let i=0; i<data['titel'].length; i++){
    if (data['titel'][i] === sel_title){
        indices.push(i);
    }
}

// Merge with existing selection
const current = new Set(source_scatter.selected.indices || []);
indices.forEach(i => current.add(i));
source_scatter.selected.indices = Array.from(current);

source_scatter.change.emit();
""")
search_input.js_on_change('value', search_callback)

# --- JS: Reset button (clears everything + zoom) ---
reset_callback = CustomJS(args=dict(
    source_scatter=source_scatter,
    source_municipalities=source_municipalities,
    dropdown=dropdown,
    search_input=search_input,
    full_data=full_data,
    p1=p1, p2=p2
), code="""
const geojson = JSON.parse(source_municipalities.geojson);
geojson.features.forEach(f => {
    f.properties.selected = false;
    f.properties.fill_color = f.properties.clickable ? 'lightblue' : 'lightgrey';
});
source_municipalities.geojson = JSON.stringify(geojson);

const data_all = full_data.data;
source_scatter.data = Object.assign({}, data_all);
source_scatter.selected.indices = [];

const all_insts = Array.from(new Set(data_all['hovedinsttx'])).sort();
dropdown.options = all_insts;
dropdown.value = all_insts[0];

search_input.value = '';
search_input.completions = Array.from(new Set(data_all['titel'])).sort();

p1.reset.emit();
p2.reset.emit();

source_scatter.change.emit();
""")
reset_button.js_on_click(reset_callback)

# --- Layout & save ---
layout = column(row(dropdown, reset_button), search_input, row(map_plot, column(p1, p2)))
output_file("dashboard_map_searchable.html")
save(layout)

print("Dashboard saved: dashboard_map_searchable.html — search integrates with selection, orange highlights, deselect works.")


C:\Users\whart\AppData\Local\Temp\ipykernel_24088\2094733013.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['instkommunetx'] = df_clean['instkommunetx'].astype(str).str.strip()
C:\Users\whart\AppData\Local\Temp\ipykernel_24088\2094733013.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['hovedinsttx'] = df_clean['hovedinsttx'].astype(str).str.strip()
C:\Users\whart\AppData\Local\Temp\ipykernel_24088\2094733013.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy o

Dashboard saved: dashboard_map_searchable.html — search integrates with selection, orange highlights, deselect works.
